import

In [18]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json
import pandas as pd

load_dotenv()

True

In [2]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

Get Spotify Token

In [3]:
def get_token():
    auth_str = client_id + ":" + client_secret
    auth_bytes = auth_str.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [4]:
token = get_token()

Authorize

In [5]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

Read file

In [10]:
spotify_list = pd.read_excel("Spotify2019-2025.xlsx")

# spotify_list.tail()

In [20]:
track_list = spotify_list["track_name"].tolist()

# track_list

Spotify Search by Track Name

In [ ]:
def search_song(query, token):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    params = {
        "q": query,
        "type": "track",
        "limit": 1
    }

    response = get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        tracks = result["tracks"]["items"]
        if tracks:
            tracks = tracks[0]
            return {
                "song_name": tracks["name"],
                "artist": tracks["artists"][0]["name"],
                "release_date": tracks["album"]["release_date"],
                "popularity": tracks["popularity"],
                "url": tracks["external_urls"]["spotify"]
            }
    return None
    

for song in track_list:
    song_info = search_song(song, token)
    if song_info:
        print(song_info)
    else:
        print("Song not found:", song)

Find Artist from Search

In [8]:
def search_for_artist(token, artist_name, offset=0, limit=50):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit={limit}&offset={offset}"
    
    query_url = url + query
    result = get(query_url, headers=headers)

    # check status code
    if result.status_code != 200:
        print(f"Error: {result.status_code} - {result.reason}")
        return None

    # parse json
    json_result = json.loads(result.content).get("artists", {}).get("items", [])
    
    return json_result

artist = search_for_artist(token, "A")

# if artist:
#     for idx, artist in enumerate(artist):
#         print(f"{idx + 1}. {artist['name']} : {artist['id']}")
# else:
#     print("No artist found")

In [ ]:
def fetch_all_artists(token, artist_name):
    artists = []
    offset = 0
    limit = 50

    while True:
        result = search_for_artist(token, artist_name, offset, limit)
        if not result:
            break

        artists.extend(result)
        offset += limit

    return artists

# search for all artists starting with "A"
artists = fetch_all_artists(token, "A")

if artists:
    for idx, artist in enumerate(artists):
        print(f"{idx + 1}. {artist['name']} : {artist['id']} : {artist['popularity']}")
else:
    print("No artist found")

Find artist's song with artist's id

In [95]:
def get_artist_song(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
    headers = get_auth_header(token)

    result = get(url, headers=headers)

    # check status code
    if result.status_code != 200:
        print(f"Error: {result.status_code} - {result.reason}")
        return None

    # parse json
    json_result = json.loads(result.content).get("tracks", [])
    
    return json_result

artist_id = artist[0]["id"]
song = get_artist_song(token, artist_id)

if song:
    for track in enumerate(song):
        print(f"{track}")

(0, {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'}, 'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR', 'id': '66CXWjxzNUsdJxJ2JdwvnR', 'name': 'Ariana Grande', 'type': 'artist', 'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'C

Find Stream's Song

In [93]:
def get_song_features(token, song_id):
    url = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = get_auth_header(token)

    result = get(url, headers=headers)

    # check status code
    if result.status_code != 200:
        print(f"Error: {result.status_code} - {result.reason}")
        return None

    # parse json
    json_result = json.loads(result.content)
    
    return json_result

song_feature = get_song_features(token, song[0]["id"])


Error: 403 - Forbidden
